# Performing photometry (automated)

In [ ]:
import warnings
from astropy.wcs import FITSFixedWarning
warnings.filterwarnings('ignore', category=FITSFixedWarning)

In [1]:
###PURPOSE: make an astropy table with all of the correct and relevant file names for each source
### we want to run photometry on

import sys

#import statements
from astropy.io import ascii
import pandas as pd
from astropy.table import join
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture
from photutils.aperture import CircularAnnulus, CircularAperture
from astropy import wcs
from photutils.aperture import SkyCircularAnnulus
from photutils.aperture import aperture_photometry
import numpy as np
import matplotlib.pyplot as plt
import aplpy
from itertools import cycle
from photutils.aperture import ApertureStats
from astropy.stats import median_absolute_deviation
from astropy.table import vstack
from astropy.table import Column
from astropy.table import join

#function to help us with plotting later on 
def fix_aplpy_fits(aplpy_obj, dropaxis=2):
    """This removes the degenerated dimensions in APLpy 2.X...
    The input must be the object returned by aplpy.FITSFigure().
    `dropaxis` is the index where to start dropping the axis (by default it assumes the 3rd,4th place).
    """
    temp_wcs = aplpy_obj._wcs.dropaxis(dropaxis)
    temp_wcs = temp_wcs.dropaxis(dropaxis)
    aplpy_obj._wcs = temp_wcs
    
scale=0.00055556*u.degree
scalearcsec=scale.to('arcsec')
length=scalearcsec.value*(19e6*u.AU)
lengthpc=length.to(u.pc)
lengtharcsec=(lengthpc/(19.3e6*u.pc))*u.arcsec
distance=19e6*u.Mpc
pclabel=(distance.value*u.AU).to(u.pc)

#read in photometry sources Google Sheet
data = ascii.read('/users/adignan/photometry_sources.csv', format='csv')

#read in Sean's catalog
df = ascii.read('/lustre/cv/students/adignan/Linden2020_table5.csv', format='csv')

minitbl=df[['Source ID','RA_J2000','Decl_J2000']]
tbl=join(data,minitbl,'Source ID')

path='/lustre/cv/students/adignan/data/'

results1=[]
medians=[]
means=[]
mads=[]
stds=[]
names1=[]
snrs=[]
beams=[]
kpcrad=[]
rins=[]
routs=[]
unscaledmad1=[]
scaledmad1=[]
errors=[]
errors90=[]

# tbl=tbl[tbl['Source ID']=='NGC2798']

for row in tbl:
    print('Working on ' + str(row['Source ID']))
    #read in relevant data files
    file3=fits.open(str(row['3 GHz file']))[0]
    file15=fits.open(str(row['15 GHz file']))[0]
    file33=fits.open(str(row['33 GHz file']))[0]
    file90 = fits.open(str(row['90 GHz file']))[0]
    weightmap = fits.open(str(row['90 GHz file']))[1]

    data3=file3.data[0,0,:,:]
    data15=file15.data[0,0,:,:]
    data33=file33.data[0,0,:,:]
    data90=file90.data[0,:,:]
    weightdata=weightmap.data

    datas=[data3,data15,data33,data90]

    wcs3=wcs.WCS(file3).dropaxis(3).dropaxis(2)
    wcs15=wcs.WCS(file15).dropaxis(3).dropaxis(2)
    wcs33=wcs.WCS(file33).dropaxis(3).dropaxis(2)
    wcs90=wcs.WCS(file90).dropaxis(2)

    wcss=[wcs3,wcs15,wcs33,wcs90]

    ras=[]
    decs=[]

    #pull out ra and dec info
    r=(row['RA_J2000'])
    r=(r[:2]+'h'+r[3:5]+'m'+r[6:]+'s')
    ras.append(r)
    d=(row['Decl_J2000'])
    d=(d[:3]+'d'+d[4:6]+'m'+d[7:]+'s')
    decs.append(d)

    positions=[]

    for r, d in zip(ras,decs):
        position=SkyCoord(ra=r,dec=d)
        positions.append(position)
        
    fwhm = row['Beam']*u.arcsec
    rin = 1.5*fwhm 
    rout = rin + fwhm

    #creating lists of the apertures and annuli (defined in sky coordinates)
    skyapertures=[]
    skyannuli=[]

    for p in positions:
        skyaperture = SkyCircularAperture(p, r=fwhm)
        skyannulus = SkyCircularAnnulus(p, rin, rout)
        skyapertures.append(skyaperture)
        skyannuli.append(skyannulus)
        
    factor=((scalearcsec.value)**2)/((np.pi/(4*np.log(2))) *(fwhm.value)**2)

    omega= (np.pi / (4*np.log(2)) ) * ((fwhm.value)**2)
    n= ((np.pi) * ((fwhm.value)**2)) / (omega)

    for ap, ann in zip(skyapertures,skyannuli):
        for d, w in zip((datas),wcss):
            phot=aperture_photometry(data=d,apertures=ap,wcs=w)
            aperstats = ApertureStats(data=d, aperture=ann, wcs=w)
            phot['pbcorr_flux_density']=(phot['aperture_sum']*factor)*1000
            phot['pbcorr_aperture_sum']=(phot['aperture_sum'])*1000
            results1.append(phot)
            medians.append(aperstats.median)
            means.append(aperstats.mean*1000)
            mads.append(aperstats.mad_std*1000)
            stds.append(aperstats.std*1000)
            names1.append(row['Source ID'])
            beams.append(row['Beam'])
            kpcrad.append( ( (row['Beam']*row['Distance']*(10**6))/(206265) ) / (1000))
            rins.append((1.5*row['Beam']))
            routs.append(((1.5*row['Beam'])+row['Beam']))
            madvalue = median_absolute_deviation(d,ignore_nan=True)
            unscaledmad1.append(madvalue*1000)
            scaledmad1.append(madvalue*(1000))
        
    table_stacked = vstack(results1)
    
    medianscol=Column(medians,name='median',unit='mJy')
    meanscol=Column(means,name='mean',unit='mJy')
    madscol=Column(mads,name='mad',unit='mJy')
    rmscol=Column(stds,name='std',unit='mJy')
    namecol1=Column(names1,name='source ID')
    beamcol=Column(beams,name='radius (arcsec)',unit='arcsec',format='{:0.3}')
    kpcradcol=Column(kpcrad,name='radius (kpc)',unit='kpc',format='{:0.2}')
    rincol=Column(rins,name='radius_in',unit='arcsec')
    routcol=Column(routs,name='radius_out',unit='arcsec')
    unscaledmadcol1=Column(unscaledmad1,name='unscaled mad entire image')
    scaledmadcol1=Column(scaledmad1,name='scaled mad entire image')

    table_stacked.add_columns([beamcol,kpcradcol,medianscol,meanscol,rmscol,madscol,unscaledmadcol1,scaledmadcol1])

    freqs = cycle([3,15,33,90])
    freqcol=Column(data=[next(freqs) for freq in range(len(table_stacked))],name='freq (GHz)')
    table_stacked.add_column(freqcol,index=1)

    table_stacked.add_column(namecol1,index=0)

    table_stacked.remove_column('id')
    table_stacked.remove_column('aperture_sum')


    print('Just finished ' + str(row['Source ID']) + '! :)')

ravals=[]
decvals=[]

for row in table_stacked:
    x1=(row['sky_center'].ra.degree)
    y1=(row['sky_center'].dec.degree)
    ravals.append(x1)
    decvals.append(y1)
    
racol=Column(data=ravals,unit='degree',name='RA (deg)')
deccol=Column(data=decvals,unit='degree',name='Dec (deg)')
# table_stacked_nonpbcorr.add_columns([racol,deccol])

results=[]
names=[]
errs=[]
mad=[]
mediansnpbc=[]
noises=[]
diffusefrac=[]

#repeat process but for non primary beam corrected images
for row in tbl:
    print('Working on ' + str(row['Source ID']))
    #read in relevant data files
    file3=fits.open(str(row['non pb corr 3 GHz file']))[0]
    file15=fits.open(str(row['non pb corr 15 GHz file']))[0]
    file33=fits.open(str(row['non pb corr 33 GHz file']))[0]
    file90 = fits.open(str(row['90 GHz file']))[0]
    weightmap = fits.open(str(row['90 GHz file']))[1]

    data3=file3.data[0,0,:,:]
    data15=file15.data[0,0,:,:]
    data33=file33.data[0,0,:,:]
    data90=file90.data[0,:,:]
    weightdata=weightmap.data

    datas=[data3,data15,data33,data90]

    wcs3=wcs.WCS(file3).dropaxis(3).dropaxis(2)
    wcs15=wcs.WCS(file15).dropaxis(3).dropaxis(2)
    wcs33=wcs.WCS(file33).dropaxis(3).dropaxis(2)
    wcs90=wcs.WCS(file90).dropaxis(2)

    wcss=[wcs3,wcs15,wcs33,wcs90]

    ras=[]
    decs=[]

    #pull out ra and dec info
    r=(row['RA_J2000'])
    r=(r[:2]+'h'+r[3:5]+'m'+r[6:]+'s')
    ras.append(r)
    d=(row['Decl_J2000'])
    d=(d[:3]+'d'+d[4:6]+'m'+d[7:]+'s')
    decs.append(d)

    positions=[]

    for r, d in zip(ras,decs):
        position=SkyCoord(ra=r,dec=d)
        positions.append(position)

    fwhm = row['Beam']*u.arcsec
    rin = 1.5*fwhm 
    rout = rin + fwhm

    #creating lists of the apertures and annuli (defined in sky coordinates)
    skyapertures=[]
    skyannuli=[]

    for p in positions:
        skyaperture = SkyCircularAperture(p, r=fwhm)
        skyannulus = SkyCircularAnnulus(p, rin, rout)
        skyapertures.append(skyaperture)
        skyannuli.append(skyannulus)

    factor=((scalearcsec.value)**2)/((np.pi/(4*np.log(2))) *(fwhm.value)**2)

    omega= (np.pi / (4*np.log(2)) ) * ((fwhm.value)**2)
    n= ((np.pi) * ((fwhm.value)**2)) / (omega)

    for ap, ann in zip(skyapertures,skyannuli):
        for d, w in zip((datas),wcss):
            phot=aperture_photometry(data=d,apertures=ap,wcs=w)
            aperstats = ApertureStats(data=d, aperture=ann, wcs=w)
            phot['nonpbcorr_flux_density']=(phot['aperture_sum']*factor)*1000
            phot['nonpbcorr_aperture_sum']=(phot['aperture_sum'])*1000
            results.append(phot)
            print(row['Source ID'])
            names.append(row['Source ID'])
            dfrac=(aperstats.median)/(phot['nonpbcorr_flux_density'])
            diffusefrac.append(dfrac)
            mediansnpbc.append(aperstats.median)
            
    for d in datas[0:1]:
    #calculate noise and errors for VLA data
        for i,j in (zip(medians[0:3],mediansnpbc[0:3])):
            madvalue=(median_absolute_deviation(d*1000,ignore_nan=True))
            vlanoise=(madvalue/(0.67))*(i)/(j)
            noises.append(vlanoise)
#                     print(vlanoise*np.sqrt(n))
            errs.append(vlanoise*np.sqrt(n))
    
    for idx,d in enumerate(datas):
        if idx == 3:
            #calculate noise and errors for GBT data
            goodidxs=((np.where(weightdata > 0)))
            x=data90[goodidxs]*np.sqrt(weightdata[goodidxs])
            kappa=( (0.67) / (median_absolute_deviation(x))) **2
            aperstats = ApertureStats(data=weightdata, aperture=skyannuli[0], wcs=wcs90)
            wghtmed=aperstats.median
            err=np.sqrt(kappa/wghtmed) * np.sqrt((np.pi*(fwhm.value)**2)/(scalearcsec.value)**2)
            errs.append(err)
            madvalue = median_absolute_deviation(data90[goodidxs])
            mad.append(madvalue*(1000))

    table_stacked_nonpbcorr = vstack(results)

    mediansnpbccol=Column(mediansnpbc,name='non PB corrected median (mJy)',unit='mJy')
    madcol=Column(mad,name='non pb corr mad entire image')
    
#     vlanoise=(mad/(0.67))*medians/mediansnpbc
    

    table_stacked_nonpbcorr.add_column([madcol])
    
    freqs = cycle([3,15,33,90])
    freqcol=Column(data=[next(freqs) for freq in range(len(table_stacked_nonpbcorr))],name='freq (GHz)')
    table_stacked_nonpbcorr.add_column(freqcol,index=1)

    namecol=Column(names,name='source ID')
    table_stacked_nonpbcorr.add_column(namecol)


    print('Just finished ' + str(row['Source ID']) + '! :)')
    
ravalsnpbc=[]
decvalsnpbc=[]

for row in table_stacked_nonpbcorr:
    x1=(row['sky_center'].ra.degree)
    y1=(row['sky_center'].dec.degree)
    ravalsnpbc.append(x1)
    decvalsnpbc.append(y1)
    
racolnpbc=Column(data=ravalsnpbc,unit='degree',name='RA (deg)')
deccolnpbc=Column(data=decvalsnpbc,unit='degree',name='Dec (deg)')
table_stacked_nonpbcorr.add_columns([racolnpbc,deccolnpbc])

errcol=Column(errs,name='error (mJy)')
table_stacked_nonpbcorr.add_column(errcol)

dfcol=Column(diffusefrac*1000,name='diffuse fraction (mJy)')
table_stacked_nonpbcorr.add_column(dfcol)

table_stacked_nonpbcorr.remove_column('id')
table_stacked_nonpbcorr.remove_column('aperture_sum')


Working on NGC0337a


Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Just finished NGC0337a! :)
Working on NGC0337b
Just finished NGC0337b! :)
Working on NGC0337c
Just finished NGC0337c! :)
Working on NGC0337d


Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 fro

Just finished NGC0337d! :)
Working on NGC0628
Just finished NGC0628! :)
Working on NGC0628Enuc.1


...

Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


NGC6946c
NGC6946c
NGC6946c
Just finished NGC6946c! :)
Working on NGC7331


Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


NGC7331
NGC7331
NGC7331
NGC7331
Just finished NGC7331! :)


ValueError: Inconsistent data column lengths

In [2]:
from astropy.table import join

# table_stacked.add_column(table_stacked_nonpbcorr['non pb corr scaled mad entire image'])
# table_stacked.add_column(table_stacked_nonpbcorr[''])
merged_table=join(table_stacked,table_stacked_nonpbcorr,keys=('source ID','freq (GHz)'))
merged_table.remove_columns(['xcenter_1','ycenter_1','sky_center_1','xcenter_2','ycenter_2','sky_center_2'])

rationew=np.divide(merged_table['pbcorr_flux_density'],merged_table['nonpbcorr_flux_density'])
newratiocol=Column(data=rationew,name='pbcorr/nonpbcorr')
# errorcol=Column(data=merged_table['non pb corr mad entire image']*np.sqrt(n)*merged_table['pbcorr_flux_density'],name='error')
# merged_table.add_columns([newratiocol,errorcol])
merged_table.rename_column('pbcorr_flux_density', 'flux density (mJy)')
merged_table.rename_column('nonpbcorr_flux_density', 'non PB corrected flux density (mJy)')
# merged_table.rename_column('non pb corr mad entire image','non PB corrected MAD (mJy/beam)')

new_order=['source ID','freq (GHz)','RA (deg)','Dec (deg)','radius (arcsec)','radius (kpc)', 
           'flux density (mJy)','non PB corrected flux density (mJy)', 'diffuse fraction (mJy)']

# merged_table[new_order].write('/users/adignan/mastertable.csv',overwrite=True)
merged_table[new_order]

KeyError: 'diffuse fraction (mJy)'

In [3]:
merged_table

source ID,freq (GHz),flux density (mJy),pbcorr_aperture_sum,radius (arcsec),radius (kpc),median,mean,std,mad,unscaled mad entire image,scaled mad entire image,id,aperture_sum,non PB corrected flux density (mJy),nonpbcorr_aperture_sum,col7,RA (deg),Dec (deg),error (mJy)
,,,,arcsec,kpc,mJy,mJy,mJy,mJy,,,,,,,,deg,deg,
str14,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64[63],float64,float64,float64
NGC0337a,3,1.7205513778781547,44.24807010181282,9.53,0.89,0.0001603786222403869,0.332031413071021,0.49764989970348195,0.11283610033899386,0.05721887282561511,0.05721887282561511,1,0.04422919466414038,1.7198174213412785,44.22919466414038,0.3566635132301599 .. 0.41110918391495943,14.958408333333331,-7.576083333333333,0.13501592488947461
NGC0337a,15,1.252183239140013,32.202869648747054,9.53,0.89,0.00014866785204503685,0.23555952583344547,0.2720863038848723,0.06407481475254288,0.10690979252103716,0.10690979252103716,1,0.03186055805068079,1.238872722085565,31.86055805068079,0.3566635132301599 .. 0.41110918391495943,14.958408333333331,-7.576083333333333,0.14092092496931474
NGC0337a,33,0.6047965207377465,15.553780718794354,9.53,0.89,1.699945096333977e-05,0.018868610739188267,0.1879614409261915,0.10039917201028158,0.10189894237555563,0.10189894237555563,1,0.014878169509076814,0.5785259106271867,14.878169509076814,0.3566635132301599 .. 0.41110918391495943,14.958408333333331,-7.576083333333333,0.18146637979194544
NGC0337a,90,0.5992092038186183,15.410089577746582,9.53,0.89,6.498488801298663e-05,0.05021175946647102,0.31259012442833134,0.31950084849466653,0.0,0.0,1,0.015410089577746582,0.5992092038186183,15.410089577746582,0.3566635132301599 .. 0.41110918391495943,14.958408333333331,-7.576083333333333,0.002467485225252586
NGC0337b,3,4.663467567533819,119.9321580854182,9.53,0.89,0.00015097580035217106,0.22900654470052834,0.255385536899418,0.21838307171046994,0.05721887282561511,0.05721887282561511,1,0.11984411713923628,4.660044164639309,119.84411713923627,0.3566635132301599 .. 0.41110918391495943,14.961179166666664,-7.582666666666666,0.13501592488947461
NGC0337b,15,2.5329128283966638,65.13987657310956,9.53,0.89,0.00011702247138600796,0.15004789714653136,0.19362638633501125,0.1898032233480022,0.10690979252103716,0.10690979252103716,1,0.06400375847166195,2.4887357702700563,64.00375847166195,0.3566635132301599 .. 0.41110918391495943,14.961179166666664,-7.582666666666666,0.14092092496931474
NGC0337b,33,1.4756634929073624,37.95019580372856,9.53,0.89,1.5951794466673164e-05,0.025891644598810274,0.13919911042862074,0.1349915229947803,0.10189894237555563,0.10189894237555563,1,0.03445832338984549,1.3398847825758784,34.45832338984549,0.3566635132301599 .. 0.41110918391495943,14.961179166666664,-7.582666666666666,0.18146637979194544
NGC0337b,90,1.5281484758635486,39.29997194816536,9.53,0.89,4.7167537559289485e-05,0.04127325893666917,0.3143429725144839,0.32175695821140926,0.0,0.0,1,0.039299971948165355,1.5281484758635486,39.29997194816536,0.3566635132301599 .. 0.41110918391495943,14.961179166666664,-7.582666666666666,0.002474131011921664


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

for i in range(1):
    for row in tbl[:1]:
        print('Working on ' + str(row['Source ID']))
        #set up the subplots
        fig = plt.figure(figsize=(15,15))
        
        #set width and height parameters for recentering
        w=0.075
        h=0.075
        #3 GHz image
        #plot the fits file
        f1 = aplpy.FITSFigure(data=str(row['3 GHz file'])[:-5]+'_copy.fits', figure=fig, subplot=[0.1,0.5,0.35,0.35])
        print(str(row['3 GHz file']))
        fix_aplpy_fits(f1)
        #pick out a colormap to use
        f1.show_colorscale(cmap='cubehelix')
        f1.add_colorbar()
        f1.colorbar.set_pad(0.25)
        #recenter based on pixel coordinates of aperture
        f1.colorbar.set_axis_label_text('mJy/beam')
        f1.recenter(table_stacked['RA'][i:i+1],table_stacked['Dec'][i:i+1],width=w, height=h)
        #plot the aperture and annulus
        f1.show_regions('/users/adignan/regfiles/'+str(row['Source ID'])+'_3GHz.reg')
        f1.add_scalebar(10*u.arcsec,label=("%.2f" % ((10*pclabel.value)/(1000)) + ' kpc'))
        f1.scalebar.set(color='white',linewidth=3) 
        f1.scalebar.set_font(size='large', weight='heavy', stretch='normal', family='serif')
        f1.axis_labels.hide()
        f1.set_title('3 GHz',fontsize=15)

        #15 GHz image
        f2 = aplpy.FITSFigure(data=str(row['15 GHz file'])[:-5]+'_copy.fits', figure=fig, subplot=[0.55,0.5,0.35,0.35])
        fix_aplpy_fits(f2)
        f2.show_colorscale(cmap='cubehelix')
        f2.add_colorbar()
        f2.colorbar.set_pad(0.25)
        f2.colorbar.set_axis_label_text('mJy/beam')
        f2.recenter(table_stacked['RA'][i+1:i+2],table_stacked['Dec'][i+1:i+2],width=w, height=h)
        f2.show_regions('/users/adignan/regfiles/'+str(row['Source ID'])+'_15GHz.reg')
        f2.add_scalebar(10*u.arcsec,label=("%.2f" % ((10*pclabel.value)/(1000)) + ' kpc'))
        f2.scalebar.set(color='black',linewidth=3)
        f2.scalebar.set_font(size='large', weight='heavy', stretch='normal', family='serif')
        f2.axis_labels.hide()
        f2.set_title('15 GHz',fontsize=15)

        #33 GHz image
        f3 = aplpy.FITSFigure(data=str(row['33 GHz file'])[:-5]+'_copy.fits', figure=fig, subplot=[0.1,0.1,0.35,0.35])
        fix_aplpy_fits(f3)
        f3.show_colorscale(cmap='cubehelix')
        f3.add_colorbar()
        f3.colorbar.set_pad(0.25)
        f3.colorbar.set_axis_label_text('mJy/beam')
        f3.recenter(table_stacked['RA'][i+2:i+3],table_stacked['Dec'][i+2:i+3],width=w, height=h)
        f3.show_regions('/users/adignan/regfiles/'+str(row['Source ID'])+'_33GHz.reg')
        f3.add_scalebar(10*u.arcsec,label=("%.2f" % ((10*pclabel.value)/(1000)) + ' kpc'))
        f3.scalebar.set(color='black',linewidth=3)
        f3.scalebar.set_font(size='large', weight='heavy', stretch='normal', family='serif')
        f3.axis_labels.hide()
        f3.set_title('33 GHz',fontsize=15)

        #90 GHz image
        f4 = aplpy.FITSFigure(data=str(row['90 GHz file'])[:-5]+'_copy.fits', figure=fig, subplot=[0.55,0.1,0.35,0.35])
        temp_wcs = f4._wcs.dropaxis(2)
        f4._wcs = temp_wcs
        f4.show_colorscale(cmap='cubehelix')
        f4.add_colorbar()
        f4.colorbar.set_pad(0.25)
        f4.colorbar.set_axis_label_text('mJy/beam')
        f4.recenter(table_stacked['RA'][i+3:i+4],table_stacked['Dec'][i+3:i+4],width=w, height=h)
        f4.show_regions('/users/adignan/regfiles/'+str(row['Source ID'])+'_90GHz.reg')
        f4.add_scalebar(10*u.arcsec,label=("%.2f" % ((10*pclabel.value)/(1000)) + ' kpc'))
        f4.scalebar.set(color='white',linewidth=3)
        f4.scalebar.set_font(size='large', weight='heavy', stretch='normal', family='serif')
        f4.axis_labels.hide()
        f4.set_title('90 GHz',fontsize=15)

        plt.suptitle(str(row['Source ID']),fontsize=25,x=0.475,y=0.9)
        
        fig.savefig('/users/adignan/figures/'+str(row['Source ID'])+'.png')

        plt.close()
        
        i=i+4

In [ ]:
for row in table_stacked[[]]:
    
    f=open('/users/adignan/regfiles/'+str(row['source ID']) + '_' + str(row['freq'])+'GHz.reg',"w")
    f.write('global color=red dashlist=8 3 width=3 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
    f.write(
        'circle'+' '+str(row['RA'].value) + ' ' + str(row['Dec'].value)+' '+str(row['radius'].value) +'"' 
        + ' #color=white' + '\n' 
        + 'circle'+' '+ str(row['RA'].value) + ' ' + str(row['Dec'].value) + ' ' + str(row['radius_in'].value) +'"' 
        + '\n' 
        + 'circle'+ ' ' + str(row['RA'].value) + ' ' + str(row['Dec'].value) + ' ' + str(row['radius_out'].value) +'"') 
    f.close()

In [ ]:
for row in tbl:
    with fits.open(row['90 GHz file']) as f:
        hdr = f[0].header 
        hdr['BSCALE']=1000.0
        f.writeto(str(row['90 GHz file'])[:-5]+'_copy.fits',overwrite=True)

# Performing photometry (manual)

In [ ]:
###PURPOSE: perform photometry for a given galaxy

#import statements
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus
from photutils.aperture import CircularAperture, CircularAnnulus
import pandas as pd
from astropy import wcs
from photutils import aperture_photometry
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization.wcsaxes import WCSAxes
from astropy.table import Table
from astropy.table import vstack
from astropy.table import Column
from photutils.aperture import ApertureStats

#read in relevant data files
jy_data = fits.open(path+'NGC2798/Jyperbeam_NGC2798_2asp_pca6_qm2_fitel_0f09-to-41f0Hz_qc_1p2rr_M_PdoCals_dt20_map_iter1.fits')[0]
# jy_image=fits.getdata(path+'NGC0337/Jyperbeam_NGC0337_2asp_pca6_qm2_fitel_0f09-to-41f0Hz_qc_1p2rr_M_PdoCals_dt20_map_iter1.fits')[0]

file3=fits.open(path+'NGC2798/ngc2798_3GHz_r0.5_smoothed_ms.fits')[0]
file15=fits.open(path+'NGC2798/ngc2798_15GHz_r0.5_smoothed_ms.fits')[0]
file33=fits.open(path+'NGC2798/ngc2798_33GHz_r0.5_smoothed_ms.fits')[0]

#making a list of the files
files=[file3,file15,file33,jy_data]

#only read in relevant axes
data3=file3.data[0,0,:,:]
data15=file15.data[0,0,:,:]
data33=file33.data[0,0,:,:]
datajy=jy_data.data[0,:,:]

#making a list of the data 
datas=[data3,data15,data33,datajy]

#drop extra non-spatial axes
wcs3=wcs.WCS(file3).dropaxis(3).dropaxis(2)
wcs15=wcs.WCS(file15).dropaxis(3).dropaxis(2)
wcs33=wcs.WCS(file33).dropaxis(3).dropaxis(2)
wcsjy=wcs.WCS(jy_data).dropaxis(2)

#making a list of the WCS info
wcss=[wcs3,wcs15,wcs33,wcsjy]

#read in Sean's catalog
df=pd.read_csv('/lustre/cv/students/adignan/Linden2020_table5.csv')
source=df[df['Source ID']==('NGC2798')]
# source=df.iloc[[143]]

#pull out positions from ra and dec columns
ra=source['RA_J2000']
dec=source['Decl_J2000']

ras=[]
decs=[]

for r in ra:
    r=(r[:2]+'h'+r[3:5]+'m'+r[6:]+'s')
    ras.append(r)
for d in dec:
    d=(d[:3]+'d'+d[4:6]+'m'+d[7:]+'s')
    decs.append(d)

#creating positions for apertures based on Sean's catalog
#defined using sky coordinates since that's what we have (ra and dec)
positions=[]

for r, d in zip(ras,decs):
    position=SkyCoord(ra=r,dec=d)
    positions.append(position)

#need area of aperture, area of beam (arcsec squared)
#divide them to get just beam
#divide this number by photometry to get in units of beam

#value of the major axis of the beam for the target
bmaj=9.528282036*u.arcsec
              
#define aperture with radius == GBT beam (bmaj value)
fwhm = bmaj
rin = 1.5*fwhm 
rout = rin + fwhm

#creating lists of the apertures and annuli (defined in sky coordinates)
skyapertures=[]
skyannuli=[]

for p in positions:
    x1, y1 = wcs3.world_to_pixel(p)
    x2, y2= wcs15.world_to_pixel(p)
    x3, y3 = wcs33.world_to_pixel(p)
    x4, y4=wcsjy.world_to_pixel(p)
    skyaperture = SkyCircularAperture(p, r=fwhm)
    skyannulus = SkyCircularAnnulus(p, rin, rout)
    skyapertures.append(skyaperture)
    skyannuli.append(skyannulus)

#creating list of the apertures defined in pixel coordinates 
#(converting from sky coordinates)
pix_apertures=[]
pix_annuli=[]

for w in wcss:
    for i,j in zip(skyapertures,skyannuli):
        pix_aperture = i.to_pixel(w)
        pix_annulus = j.to_pixel(w)
        pix_apertures.append(pix_aperture)
        pix_annuli.append(pix_annulus)
    
factor=(np.pi*((fwhm.value)**2))/(np.pi/4/np.log(2) * bmaj.value)

# actually performing photometry
# lists=Table(names='aperture_sum')
#initialize empty photometry table   
# for s in skyapertures:
#     for d,w in zip(datas,wcss):
#         row = aperture_photometry(data=d, apertures=s, wcs=w)
#         row['aperture_sum']=row['aperture_sum']/factor 
#         print(row)

results=[]
medians=[]
means=[]
mads=[]
rmss=[]

for ap, ann in zip(skyapertures,skyannuli):
    for d, w in zip((datas),wcss):
        phot=aperture_photometry(data=d,apertures=ap,wcs=w)
        aperstats = ApertureStats(data=d, aperture=ann, wcs=w)
        phot['aperture_sum']=(phot['aperture_sum']/factor)*1000
        results.append(phot)
        medians.append(aperstats.median*1000)
        means.append(aperstats.mean*1000)
        mads.append(aperstats.mad_std*1000)
        rmss.append(np.sqrt(aperstats.mean*1000))

table_stacked = vstack(results)
# table_stacked |= (d)
medianscol=Column(medians,name='median',unit='mJy')
meanscol=Column(means,name='mean',unit='mJy')
madscol=Column(mads,name='mad',unit='mJy')
rmscol=Column(rmss,name='rms',unit='mJy')

table_stacked.add_columns([medianscol,meanscol,madscol,rmscol])

table_stacked['aperture_sum']=Column(data=table_stacked['aperture_sum'],unit='mJy')

# name = Column(, name='freq')
# table_stacked.add_column(freq,index=0)

#read in photometry sources Google Sheet
data = ascii.read('/users/adignan/photometry_sources.csv', format='csv')

#read in Sean's catalog
df = ascii.read('/lustre/cv/students/adignan/Linden2020_table5.csv', format='csv')

minitbl=df[['Source ID','RA_J2000','Decl_J2000']]
tbl=join(data,minitbl,'Source ID')

for row in tbl:
    print(row['Source ID'])

table_stacked
  

# Making photometry figures 

# This code works; don't touch it!

In [ ]:
import aplpy
import matplotlib.pyplot as mpl
from astropy import wcs
from astropy.io import fits

#read in relevant data files
jy_data = fits.open(path+'NGC0337/Jyperbeam_NGC0337_2asp_pca6_qm2_fitel_0f09-to-41f0Hz_qc_1p2rr_M_PdoCals_dt20_map_iter1.fits')[0]
# jy_image=fits.getdata(path+'NGC0337/Jyperbeam_NGC0337_2asp_pca6_qm2_fitel_0f09-to-41f0Hz_qc_1p2rr_M_PdoCals_dt20_map_iter1.fits')[0]

file3=fits.open(path+'NGC0337/ngc0337_3GHz_r0.5_smoothed_ms.pbcor.gbt_resmatched_regridded.fits')[0]
file15=fits.open(path+'NGC0337/ngc0337_15GHz_r0.5_smoothed_ms.pbcor.gbt_resmatched_regridded.fits')[0]
file33=fits.open(path+'NGC0337/ngc0337_33GHz_r0.5_smoothed_ms.pbcor.gbt_resmatched_regridded.fits')[0]

#making a list of the files
files=[file3,file15,file33,jy_data]

#only read in relevant axes
data3=file3.data[0,0,:,:]
data15=file15.data[0,0,:,:]
data33=file33.data[0,0,:,:]
datajy=jy_data.data[0,:,:]

fig = plt.figure(figsize=(15,15))

f1 = aplpy.FITSFigure(data3, figure=fig, subplot=[0.1,0.5,0.35,0.35])
f1.show_colorscale(cmap='cubehelix')
f1.recenter(x1,y1,radius=50)
# pix_apertures[0].plot(color='white',lw=2)
# pix_annuli[0].plot(color='red', lw=2)

f2 = aplpy.FITSFigure(data15, figure=fig, subplot=[0.5,0.5,0.35,0.35])
f2.show_colorscale(cmap='cubehelix')
f2.recenter(x2,y2,radius=50)
# pix_apertures[1].plot(color='white',lw=2)
# pix_annuli[1].plot(color='red', lw=2)
# plt.text(file15.header['CRPIX1'],115,s='15 GHz',color='white',fontsize='xx-large',horizontalalignment='center',verticalalignment='baseline')

f3 = aplpy.FITSFigure(data33, figure=fig, subplot=[0.1,0.1,0.35,0.35])
f3.show_colorscale(cmap='cubehelix')
f3.recenter(x3,y3,radius=50)
# pix_apertures[2].plot(color='white',lw=2)
# pix_annuli[2].plot(color='red', lw=2)
# plt.text(file33.header['CRPIX1'],115,s='33 GHz',color='black',fontsize='xx-large',horizontalalignment='center',verticalalignment='baseline')

f4 = aplpy.FITSFigure(datajy, figure=fig, subplot=[0.5,0.1,0.35,0.35])
f4.show_colorscale(cmap='cubehelix')
f4.recenter(x4,y4,radius=50)
# pix_apertures[3].plot(color='white',lw=2)
# pix_annuli[3].plot(color='red', lw=2)

# plt.text(file3.header['CRPIX1'],115,s='90 GHz',color='white',fontsize='xx-large',horizontalalignment='center',verticalalignment='baseline')

# plt.suptitle(str(row['Source ID']),fontsize=25,x=0.475,y=0.88)

# Test code to figure out scale bar

In [ ]:
import aplpy
import matplotlib.pyplot as mpl
from astropy import wcs
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy import coordinates

fig = plt.figure(figsize=(15,15))
f1 = aplpy.FITSFigure(data3, figure=fig, subplot=[0.1,0.5,0.35,0.35])
f1.show_colorscale(cmap='cubehelix')
f1.recenter(x=x1,y=y1,radius=50)
scalebar_right = coordinates.SkyCoord(x1,y1, frame='fk5')
length = (0.5*u.pc / (5400*u.pc)).to(u.deg, u.dimensionless_angles())
plt.plot([scalebar_right.ra.deg, (scalebar_right.ra-length).deg]*u.deg,
        [(scalebar_right.dec).deg]*2*u.deg,
        'k',
        zorder=100, linewidth=2)
plt.text((scalebar_right.ra-length/2).deg, (scalebar_right.dec+0.001*u.deg).deg,
        "0.5 pc", color='k', ha='center')
pix_apertures[0].plot(color='white',lw=2)
# plt.text(file3.header['CRPIX1'],115,s='3 GHz',color='white',fontsize='xx-large',horizontalalignment='center',verticalalignment='baseline')
# print(proj_plane_pixel_scales(wcs3))
# f1.add_scalebar(1/60.)
pix_scale = proj_plane_pixel_scales(wcs3)

## Performing photometry for NGC0337a

## Just one source here, baby steps

In [ ]:
###PURPOSE: perform photometry on each of the sources in NGC0337

#import statements
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture
from photutils.aperture import CircularAnnulus, CircularAperture
import pandas as pd
from astropy import wcs
from photutils import SkyCircularAnnulus
from photutils import aperture_photometry
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#read in relevant data files
jy_data = fits.open(path+'NGC0337/Jyperbeam_NGC0337_2asp_pca6_qm2_fitel_0f09-to-41f0Hz_qc_1p2rr_M_PdoCals_dt20_map_iter1.fits')

file3=fits.open(path+'NGC0337/ngc0337_15GHz_r0.5_smoothed_ms.pbcor.gbt_resmatched_regridded.fits')[0]
file15=fits.open(path+'NGC0337/ngc0337_3GHz_r0.5_smoothed_ms.pbcor.gbt_resmatched_regridded.fits')[0]
file33=fits.open(path+'NGC0337/ngc0337_33GHz_r0.5_smoothed_ms.pbcor.gbt_resmatched_regridded.fits')[0]

files=[file3,file15,file33]

data3=file3.data[0,0,:,:]
data15=file15.data[0,0,:,:]
data33=file33.data[0,0,:,:]

#removing nan values from data arrays for future use
data3_masked=data3[~np.isnan(data3)]

datas=[data3,data15,data33]

#drop extra non-spatial axes
wcs3=wcs.WCS(file3).dropaxis(3).dropaxis(2)
wcs15=wcs.WCS(file15).dropaxis(3).dropaxis(2)
wcs33=wcs.WCS(file33).dropaxis(3).dropaxis(2)

wcss=[wcs3,wcs15,wcs33]

#read in Sean's catalog
df=pd.read_csv('/lustre/cv/students/adignan/Linden2020_table5.csv')
#df[df['Source ID'].str.contains("0337")]
#pick one source to focus on for now
source=df[df['Source ID'].str.contains('0337a')]

#pull out position from ra and dec
ra=source['RA_J2000'][0]
dec=source['Decl_J2000'][0]
for r in ra:
    (ra[:1]+'h'+ra[3:4]+'m'+ra[6:]+'s')
# 00 59 50.018
#0h59m50.018s
# -07 34 33.9
#-7d34m33.9s

#set center of aperture to position from Linden2020
position=SkyCoord(ra='0h59m50.018s',dec='-7d34m33.9s')

#define aperture with radius == GBT beam (bmaj value)
fwhm = 9.528282036*u.arcsec
rin = 1.5*fwhm 
rout = rin + fwhm
skyaperture = SkyCircularAperture(position, r=fwhm)
skyannulus = SkyCircularAnnulus(position, rin, rout)
pix_aperture = [skyaperture.to_pixel(w) for w in wcss]
# ap = CircularAperture(position, r=fwhm)
# ann = CircularAnnulus(position, rin, rout)
phot_table=[]
# now, need to measure the aperture sums ect and then plot those! 
annuli = [skyannulus.to_pixel(wcs) for wcs in wcss]
for d, w in zip(datas,wcss):
    row = aperture_photometry(data=d, apertures=skyaperture, wcs=w)
    phot_table.append(row['aperture_sum'])
    
# phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output
print(phot_table)


# Miscellaneous cell to try to fix cut-off images using np.pad (didn't work)

In [ ]:
newarr=np.pad(array=datajy,pad_width=300,constant_values=0)
hdu = fits.PrimaryHDU(newarr)
hdul = fits.HDUList([hdu])
fits.writeto(filename='newarr.fits',data=newarr,header=jy_data.header)